<a href="https://colab.research.google.com/github/Navam16/class-participation/blob/main/Students_participation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install openai-whisper sentence-transformers

# Import libraries
import whisper
import os
from google.colab import files
from sentence_transformers import SentenceTransformer, util
from termcolor import colored  # To enhance output readability

# Load Models
whisper_model = whisper.load_model("base")  # Whisper for speech-to-text
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")  # SBERT for similarity scoring


In [ ]:
def upload_audio(role):
    """Prompts the user to upload an audio file and returns the file path."""
    print(colored(f"\n🎤 Upload the {role}'s audio:", "cyan", attrs=["bold"]))
    uploaded = files.upload()
    for filename in uploaded.keys():
        return filename

def transcribe_audio(audio_path):
    """Transcribes audio using Whisper and returns text."""
    print(colored("🔍 Transcribing audio...", "yellow"))
    result = whisper_model.transcribe(audio_path)
    return result["text"]


In [ ]:
# Upload & Transcribe Professor's Audio
prof_audio_path = upload_audio("Professor Audio")
professor_text = transcribe_audio(prof_audio_path)
print(colored("\n📚 Professor's Transcription:", "blue", attrs=["bold"]))
print(colored(professor_text, "light_blue"))

# Upload & Transcribe Student's Audio
student_audio_path = upload_audio("student audio")
student_text = transcribe_audio(student_audio_path)
print(colored("\n🗣️ Student's Transcription:", "green", attrs=["bold"]))
print(colored(student_text, "light_green"))


In [ ]:
def calculate_relevance(prof_text, student_text):
    """Calculates semantic similarity score between professor's and student's text."""
    print(colored("\n🔍 Evaluating Student's Relevance...", "yellow"))
    prof_embedding = sbert_model.encode(prof_text, convert_to_tensor=True)
    student_embedding = sbert_model.encode(student_text, convert_to_tensor=True)
    score = util.pytorch_cos_sim(prof_embedding, student_embedding).item()
    return round(score * 100, 2)  # Convert to percentage

# Compute Score
relevance_score = calculate_relevance(professor_text, student_text)

# Display Final Score with Remarks
print(colored("\n📊 Student's Relevance Score: ", "magenta", attrs=["bold"]), end="")
print(colored(f"{relevance_score}/100", "light_magenta", attrs=["bold"]))

# Provide feedback based on score
if relevance_score > 85:
    print(colored("\n🌟 Excellent! The student stayed highly relevant to the topic.", "green", attrs=["bold"]))
elif 60 <= relevance_score <= 85:
    print(colored("\n✅ Good effort! The student covered most of the key points.", "yellow", attrs=["bold"]))
else:
    print(colored("\n⚠️ Needs Improvement! The response lacked alignment with the topic.", "red", attrs=["bold"]))
